## Using Python 2.7

In [ ]:
from __future__ import print_function

In [ ]:
import torch as tc
import torchvision as tcv
import torchvision.transforms as transforms

In [ ]:
dataPath = '../../Datasets/pytorch'
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = tcv.datasets.CIFAR10(root=dataPath, train=True, download=True, transform=transform)
trainloader = tc.utils.data.DataLoader(trainset, batch_size=4, shuffle=False, num_workers=2)
testset = tcv.datasets.CIFAR10(root=dataPath, train=False, download=True, transform=transform)
testloader = tc.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
#show some random images
import matplotlib.pyplot as plt
import numpy as np

def imshow(img):
    img = img / 2 + 0.5   #unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
    
dataiter = iter(trainloader)
images, labels = dataiter.next()

imshow(tcv.utils.make_grid(images))

print(' '.join('%5s' % classes[labels[j]] for j in range(4)))


In [ ]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(2):
    
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        
        inputs, labels = Variable(inputs), Variable(labels)
        
        optimizer.zero_grad()
        
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.data[0]
        if(i % 2000 == 1999) :
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
            
print("Finished Training")

In [ ]:
testiter = iter(testloader)
testimages, labels = testiter.next()

imshow(tcv.utils.make_grid(testimages))

print(' '.join("%5s" % classes[labels[j]] for j in range(4)))

In [ ]:
outputs = net(Variable(testimages))
_, predicted = tc.max(outputs.data, 1)

print("Predicted: ", ' '.join("%5s" % classes[predicted[j]] for j in range(4)))

In [ ]:
# See how our network performed
correct = 0
total = 0
for data in testloader:
    images, labels = data
    outputs = net(Variable(images))
    _, predicted = tc.max(outputs, 1)
    total += labels.size(0)
    correct += (labels == predicted.data).sum()
    
print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))